## FDW Crop Production Data Profiling - Nigeria

In [1]:
import os, sys, glob, json
from itertools import product, compress, chain
from functools import reduce
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import requests
import numpy as np
import pandas as pd
import geopandas as gpd
from tools import save_hdf, save_npz, load_npz, PrintAdminUnits, PlotAdminShapes
from tools import FDW_PD_Sweeper, FDW_PD_AvalTable, FDW_PD_Compiling, FDW_PD_GrainTypeAgg, FDW_PD_ValidateFnidName
from tools import FDW_PD_CreateAdminLink, FDW_PD_RatioAdminLink, FDW_PD_ConnectAdminLink
from tools import FDW_PD_CaliSeasonYear
from tools_graphic import PlotBarProduction, PlotLinePAY, PlotHeatCropSystem, PlotHeatSeasonData
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
pd.options.mode.chained_assignment = None

In [2]:
# CPCV2 grain code ------------------------------ #
grain_code = pd.read_hdf('../data/crop/grain_cpcv2_code.hdf')
product_category = grain_code[['product', 'product_category']].set_index('product').to_dict()['product_category']
# ----------------------------------------------- #

# Load FEWS NET administrative boundaries ------- #
epsg = 'epsg:26331' # Nigeria
fn_shapes = sorted(glob.glob('../data/shapefile/fewsnet/NG_Admin?_????.shp'))
shape_all = []
for fn in fn_shapes:
    name = fn[-18:-4]
    exec('%s = gpd.read_file("%s").to_crs("%s")' % (name, fn, epsg))
    exec('%s["area"] = %s["geometry"].area/10**6' % (name, name))
    exec('shape_all.append(%s)' % (name))
shape_all = pd.concat(shape_all, axis=0).reset_index(drop=True)
PrintAdminUnits(shape_all)
# ----------------------------------------------- #

# FDW API host address -------------------------- #
host = 'https://fdw.fews.net'
auth = tuple(json.loads(open('token.json', "r").read()))
parameters = {
    'format': 'json',
    'country': 'Nigeria',
    'product': ['R011','R012','R013','R014','R015','R016','R017','R018','R019'],
    'survey_type': 'crop:all'
}
endpoint = '/api/cropproductionindicatorvalue/'
response = requests.get(host + endpoint, auth=auth, params=parameters, proxies={})
response.raise_for_status()
df = pd.DataFrame.from_records(response.json())
df_origin = df.copy()
# ----------------------------------------------- #

- FEWS NET admin shapefiles ------------------- #
| year	 | Admin1   | # units   | Admin2   | # units   | Admin3   | # units   |
| 1991	 | NG1991A1 | 31	| nan	| 0	| nan	| 0	|
| 1996	 | NG1996A1 | 37	| nan	| 0	| nan	| 0	|
| 2014	 | NG2014A1 | 37	| NG2014A2	| 774	| nan	| 0	|
----------------------------------------------- #


In [3]:
# Manual Pre-processing before Sweeping --------- #
# 1. Default setting 
# a) None-type crop production system
df.loc[df['crop_production_system'].isna(), 'crop_production_system'] = 'none'
df.loc[df['crop_production_system'] == '', 'crop_production_system'] = 'none'
# b) None-type population group
df.loc[df['population_group'].isna(), 'population_group'] = 'none'
df.loc[df['population_group'] == '', 'population_group'] = 'none'

# ----------------------------------------------- #

# FDW Production Data Inspection ---------------- #
df, df_raw = FDW_PD_Sweeper(df, area_priority='Area Planted')
table_dict = FDW_PD_AvalTable(df, shape_all)
# ----------------------------------------------- #

# FEWS NET Shapefile comparison ----------------- #
shape_used = pd.concat([NG_Admin1_1996], axis=0)
PlotAdminShapes(shape_used, label=True)
# ----------------------------------------------- #s

- Remove missing records ---------------------- #
Orignial data points: 38,603
Removed 8,320 "Missing Value" points
0/6,080 "Area Harvested" points are retained.
9,819/10,841 "Area Planted" points are retained.
10,743/10,841 "Quantity Produced" points are retained.
9,721/10,841 "Yield" points are retained.
Current data points: 30,283



AssertionError: 

In [ ]:
# Remove incomplete records -------------------------- #
# Generate virtual IDs to each set of ['fnid','crop_production_system','season_year', 'product'].
df['vid'] = 0
indicators = ['Area Harvested', 'Area Planted', 'Quantity Produced', 'Yield']
vid_dict = dict()
records = df[['fnid','crop_production_system','season_year','product']].drop_duplicates().reset_index(drop=True)
records[indicators] = np.nan

bad = []

for i, row in records.iterrows():
    fnid, crop_production_system, season_year, product_name = row[['fnid','crop_production_system','season_year','product']]
    sub = df[
        (df['fnid'] == fnid) &
        (df['crop_production_system'] == crop_production_system) &
        (df['season_year'] == season_year) &
        (df['product'] == product_name)
    ]
    # Ignore if either area, production, and yield is missing
    n_points = sub.shape[0]
    if n_points < 3: continue
    if n_points > 4: 
        bad.append(sub)

- Nigeria crop seasonal calendar

![FEWS NET](../figures/crop_calendar/seasonal-calendar-nigeria.png)



<img src="../figures/crop_calendar/Nigeria_-_Crop_calendar.jpg"  width="60%">

- FDW data consists of `NG1996A1`.

| Year  | Admin-1  | # units  | Admin-2  | # units |
| :---: | :----:   | :----:   | :----:   | :---:   |
| 1991  | NG1991A1 | 31       | -        | -       |
| 1996  | NG1996A1 | 37       | -        | -       |
| 2014  | NG2014A1 | 37       | NG2014A2 | 774     |

- Comparison between admin boundaries.

![image](https://github.com/chc-ucsb/GlobalCropData/blob/main/figures/NG_admin_shapes.png?raw=true)

- **`NG1996A1`** is used to represent the current admin-level 1 crop data.
- Nigeria has a single crop season: `Wet`.
- Nigeria has no population group(s).

In [ ]:
# Define the latest shapefile ------------------- #
latest_level = 1
shape_latest = NG_Admin1_1996.copy().to_crs('epsg:4326')
# ----------------------------------------------- #

# Validation of FNIDs and Names ----------------- #
df = FDW_PD_ValidateFnidName(df, shape_used, shape_latest)
# ----------------------------------------------- #

# FDW Production Data Compiling ----------------- #
area, prod = FDW_PD_Compiling(df, shape_used)
area_all, prod_all = area.copy(), prod.copy()
mdx_pss = area.columns.droplevel([0,1]).unique()
# ----------------------------------------------- #

In [ ]:
# Link admin boundaries ------------------------- #
link, over = FDW_PD_CreateAdminLink(BF_Admin1_1984, BF_Admin2_2001, 'ADMIN1', 'ADMIN2', area, epsg)
# Manual Editing
link['BF1984A169'].update(method='ABR')
link['BF1984A155'].update(method='ABR', fnids={'BF2001A20502': {'name': 'Namentenga'}})
link['BF1984A152'].update(method='ABR')
link['BF1984A173'].update(method='ABR', fnids={'BF2001A20603': {'name': 'Sissili'},'BF2001A20604': {'name': 'Ziro'}})
# Crop specific ratios
link_ratio = FDW_PD_RatioAdminLink(link, area, over, mdx_pss)
# Add current unit to link_ratio
for fnid_new in link_ratio.keys():
    link_ratio[fnid_new][fnid_new] = 1.0
    link_ratio[fnid_new] = link_ratio[fnid_new].sort_index(axis=1, ascending=False)
# Connect data with AdminLink
area_new, prod_new = FDW_PD_ConnectAdminLink(link_ratio, area, prod, validation=True)
# ----------------------------------------------- #

# Aggregate grain data by grain type ------------ #
[area_new, prod_new, area_all, prod_all] = FDW_PD_GrainTypeAgg([area_new, prod_new, area_all, prod_all], product_category)
# ----------------------------------------------- #

# Manual correction ----------------------------- #
crop_new = prod_new/area_new
# ----------------------------------------------- #

# Complete long format DataFrame
df_area = area_new.T.stack().reset_index().rename({0:'value'},axis=1)
df_area['indicator'] = 'area'
df_prod = prod_new.T.stack().reset_index().rename({0:'value'},axis=1)
df_prod['indicator'] = 'production'
df_yield = (prod_new/area_new).T.stack().reset_index().rename({0:'value'},axis=1)
df_yield['indicator'] = 'yield'
stack = pd.concat([df_area, df_prod, df_yield], axis=0)
# Insert a country name
stack['country'] = 'Burkina Faso'
stack = stack[['fnid','country','name','product','year','season_name','season_date','indicator','value']]
stack = stack.reset_index(drop=True)
# Change season_date to harvest_end of the FEWS NET's crop calendar
stack.rename(columns={'season_date':'harvest_end'},inplace=True)
stack['harvest_end'] = stack['harvest_end'].replace({
    '10-01':'12-01' # Main
})

# Save data
save_hdf('./data/crop/adm_crop_production_raw_BF.hdf', df)
save_hdf('./data/crop/adm_crop_production_BF.hdf', stack)
save_npz('./data/crop/adm_crop_production_BF_ratio.npz', link_ratio)

## Visualization of production data

In [ ]:
# Bar chart of national grain production
country_iso, country_name = 'BF', 'Burkina Faso'
df = pd.read_hdf('./data/crop/adm_crop_production_%s.hdf' % country_iso)
year = [df['year'].min(), df['year'].max()]
product_order = ['Sorghum','Millet','Maize','Rice','Fonio']
for season_name in ['Main']:
    footnote = 'National grain production in %s - %s' % (country_name, season_name)
    fn_save = './figures/%s_bar_natgrainprod_%s.png' % (country_iso, season_name)
    sub = df[df['season_name'] == season_name]
    fig = PlotBarProduction(sub, year, product_order, footnote, fn_save)
    # fig.show()

![image](https://github.com/chc-ucsb/GlobalCropData/blob/main/figures/BF_bar_natgrainprod_Main.png?raw=true)

In [ ]:
# Lineplot of Production-Area-Yield (PAY) time-series
country_iso, country_name = 'BF', 'Burkina Faso'
df = pd.read_hdf('./data/crop/adm_crop_production_%s.hdf' % country_iso)
year = [df['year'].min(), df['year'].max()]
product_season = [
    ['Maize','Main'],
    ['Sorghum','Main'],
    ['Millet','Main'],
]
for product_name, season_name in product_season:
    footnote = 'Production-Area-Yield (PAY) time-series of %s - %s - %s' % (country_iso, product_name, season_name)
    fn_save = './figures/%s_line_pay_%s_%s.png' % (country_iso, product_name, season_name)
    sub = df[(df['product'] == product_name) & (df['season_name'] == season_name)]
    fig = PlotLinePAY(sub, year, footnote, fn_save)
    # fig.show()

![image](https://github.com/chc-ucsb/GlobalCropData/blob/main/figures/BF_line_pay_Maize_Main.png?raw=true)
![image](https://github.com/chc-ucsb/GlobalCropData/blob/main/figures/BF_line_pay_Sorghum_Main.png?raw=true)
![image](https://github.com/chc-ucsb/GlobalCropData/blob/main/figures/BF_line_pay_Millet_Main.png?raw=true)

In [ ]:
df = pd.read_hdf('./data/crop/adm_crop_production_raw_%s.hdf' % country_iso)
df['product'].unique()

In [ ]:
# Heatmap of seasonal data availability
country_iso, country_name = 'BF', 'Burkina Faso'
df = pd.read_hdf('./data/crop/adm_crop_production_raw_%s.hdf' % country_iso)
code = {'Main':1}
comb = {1:1}
comb_name = {1:'Main'}
for product_name in ['Maize (Corn)','Millet','Sorghum']:
    data = df[(df['product'] == product_name) & (df['season_name'].isin(code.keys()))]
    footnote = 'Seasonal data availability in %s - %s (uncorrected)' % (country_name, product_name)
    fn_save = './figures/%s_heat_seasondata_%s.png' % (country_iso, product_name)
    fig = PlotHeatSeasonData(data, code, comb, comb_name, footnote, fn_save)
    fig.show()

![image](https://github.com/chc-ucsb/GlobalCropData/blob/main/figures/BF_heat_seasondata_Maize%20(Corn).png?raw=true)
![image](https://github.com/chc-ucsb/GlobalCropData/blob/main/figures/BF_heat_seasondata_Millet.png?raw=true)
![image](https://github.com/chc-ucsb/GlobalCropData/blob/main/figures/BF_heat_seasondata_Sorghum.png?raw=true)

In [ ]:
# Calibrated PAY time-series per FNID
from tools_graphic import PlotLineCropTS
country_iso, country_name = 'BF', 'Burkina Faso'
df = pd.read_hdf('./data/crop/adm_crop_production_%s.hdf' % country_iso)
link_ratio = load_npz('./data/crop/adm_crop_production_%s_ratio.npz' % country_iso)
year_all = np.arange(df['year'].min(), df['year'].max()+1)
prod_season = [
    ['Maize', 'Main'],
    ['Sorghum', 'Main']
]
for product_name, season_name in prod_season:
    sub = df[
        (df['product'] == product_name) &
        (df['season_name'] == season_name)
    ]
    for fnid in sub['fnid'].unique():
        sub_fps = sub[sub['fnid'] == fnid]
        fn_save = './figures/crop_calibrated/%s_%s_%s_%s.png' % (country_iso, product_name, season_name, fnid)
        # fig = PlotLineCropTS(sub_fps, fnid, product_name, season_name, link_ratio, year_all, fn_save)